## Setup

In [ ]:
# %%bash
# !(stat -t /usr/local/lib/*/dist-packages/google/colab > /dev/null 2>&1) && exit 
# rm -rf jem-3-bert
# git clone https://github.com/jfanghrv/jem-3-bert.git

In [ ]:
# ! pip install datasets
# ! pip install transformers

In [23]:
import json 
import os
from tqdm import tqdm
import csv

from datasets import load_dataset
import torch
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM

import pandas as pd
import numpy as np
import re

import matplotlib
import matplotlib.pyplot as plt

## Data Preprocessing

For model training and inference, PICARD needs the table names and corresponding column names in each database. 

`[question] | [db_id] | [table] : [column] ( [content] , [content] ) , [column] ( ... ) , [...] | [table] : ... | ...`

An example input looks like this:

`How many singers do we have? | concert_singer | stadium : stadium_id, location, name, capacity, highest, lowest, average | singer : singer_id, name, country, song_name, song_release_year, age, is_male | concert : concert_id, concert_name, theme, stadium_id, year | singer_in_concert : concert_id, singer_id`


In [14]:
data_path = './data/tables.json'
with open(data_path) as f:
    data_json = f.read()
data = json.loads(data_json)

In [15]:
dbs = dict()
for db in data:
    table_cols = [[] for _ in range(len(db['table_names']))]
    for i, colname in db['column_names_original']:
        if i >= 0:
            table_cols[i].append(colname.lower())
    table_cols = [ table + ' : ' + ', '.join(cols) for cols, table in zip(table_cols, db['table_names']) ] 
    dbs[db['db_id']] = ' | '.join(table_cols)

In [16]:
# load dataset
train_ds, val_ds, test_ds = load_dataset('spider', split=['train[:5000]', 'train[5000:7000]','validation[:]'])

Generating train split:   0%|          | 0/7000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1034 [00:00<?, ? examples/s]

Dataset spider downloaded and prepared to /Users/erg1/.cache/huggingface/datasets/spider/spider/1.0.0/4e5143d825a3895451569c8b9b55432b91a4bc2d04d390376c950837f4680daa. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

## PICARD Model

The base model we use is PICARD pretrained on Spider dataset. 

In [5]:
tokenizer = AutoTokenizer.from_pretrained("tscholak/1zha5ono")

In [6]:
# model = AutoModelForSeq2SeqLM.from_pretrained("tscholak/cxmefzzi")
model = AutoModelForSeq2SeqLM.from_pretrained("tscholak/1zha5ono")

In [ ]:
# model.eval()
# for i in tqdm(range(10)):
#     input_ = test_ds[i]
#     input_str = f"{input_['question']} | {input_['db_id']} | {dbs[input_['db_id']]}"
#     token_out = tokenizer([input_str], return_tensors='pt')
#     model_out = model.generate(token_out['input_ids'])

#     print(input_['question'])
#     print('Truth:', ' '.join(input_['query_toks_no_value']))
#     print('Model Output:',tokenizer.batch_decode(model_out, skip_special_tokens=True)[0].split(' | ')[1])
#     print()
    
    

## Run Inference

In [ ]:
model.eval()
out_eval = {}
eval_df = val_ds
write_to_file = True # whether to write resulting inference dataframe to file
inference_data_path = 'data/spider_val_ds_eval.csv' # if writing df to file, this is the file path
for i in tqdm(range(len(eval_df))):
    input_ = test_ds[i]
    input_str = f"{input_['question']} | {input_['db_id']} | {dbs[input_['db_id']]}"
    token_out = tokenizer([input_str], return_tensors='pt')
    #model_out = model.generate(token_out['input_ids'])
    generation_output = model.generate(
                    input_ids=token_out['input_ids'],
                    return_dict_in_generate=True, 
                    output_scores=True
                    )
    preds = generation_output['sequences'].cpu() 
    sequences_scores = generation_output['sequences_scores'].cpu() 
    logits = torch.stack(generation_output['scores'], dim=1).cpu()
    output_prob = torch.softmax(logits, dim=2)
    log_prob = torch.log_softmax(logits, dim=2)
    sequences_entropy = ( torch.sum(output_prob * log_prob, dim=2) * (-1) ).numpy()
    
        
    #text = tokenizer.decode(token_out['input_ids'][j], skip_special_tokens=True)
    pred = tokenizer.decode(preds[0], skip_special_tokens=True).split(' | ')[1]


    pred_tensor = preds[1:]
    entropy = sequences_entropy[0].tolist()
    if tokenizer.eos_token_id in pred_tensor:
        pred_eos_idx = torch.nonzero(pred_tensor==tokenizer.eos_token_id)[0].item()
        entropy = entropy[:pred_eos_idx+1]
    result = {}
    result['question'] = input_['question']
    result['real'] = input_['query']
    result['pred'] = pred
    result['sequence_entropy'] = [list(np.round(entropy,6))]
    out_eval[i] = result

reformed_dict = {'question': [], 'real': [], 'pred': [],'sequence_entropy': []}
for outerKey, innerDict in out_eval.items():
    for innerKey, values in innerDict.items():
        reformed_dict[innerKey].append(values)
inference_df = pd.DataFrame(reformed_dict)

if write_to_file:
    inference_df.to_csv(inference_data_path,index=False)

## Create Evaluation Files

In [52]:
#inference_df = pd.read_csv('data/spider_val_ds_eval.csv')
inference_data = 'data/spider_val_ds_eval.csv' # if writing df to file, this is the file path
eval_df = val_ds
dest_path = 'evaluation/spider_val' # do not include file type
def write_eval_files(inference_data, db_ids, dest_path):
    '''
    Creates gold.txt and predict.txt files in the format that spider evaluation suite wants
    df_path: path to the dataset containing results from running inference (see Run Inference section above)
    db_ids: the db_ids, which are needed to create the txt files. Example: eval_df['db_id']
    '''
    if type(inference_data) == str:
        eval_df = pd.read_csv(inference_data)
    else:
        eval_df = inference_data.copy()
    eval_df['real'] = eval_df['real'].apply(lambda x: x.replace("\t",''))
    eval_df['real_with_db_id'] = eval_df.real + "\t" + db_ids
    eval_df['real_with_db_id'].to_csv(dest_path+'_gold.txt', header=False, index=False, quoting=csv.QUOTE_NONE,sep='\n')
    (eval_df.index.astype(str) + ' ' + eval_df.pred).to_csv(dest_path+'_predict.txt', header=False, index=False, quoting=csv.QUOTE_NONE,sep='\n')



In [64]:
t = [['0 sd bfb'],['1 gdg fdgs'],['12 adggf nf']]
[int(i[0][:i[0].find(' ')]) for i in t]

[0, 1, 12]

In [63]:
[[i[0][i[0].find(' ')+1:]] for i in t]

[['sd bfb'], ['gdg fdgs'], ['adggf nf']]

In [53]:
write_eval_files(inference_data, eval_df['db_id'], dest_path)

inference_data = 'data/spider_test_ds_eval.csv' 
eval_df = test_ds
dest_path = 'evaluation/spider_test' # do not include file type

write_eval_files(inference_data, eval_df['db_id'], dest_path)

## Entropy Thresholding

In [6]:
val_inference_df = pd.read_csv('data/spider_val_ds_eval.csv')
test_inference_df = pd.read_csv('data/spider_test_ds_eval.csv')

In [7]:
def calc_max_entropy(sequence_entropy):
    max_entropy_l = []
    for i in sequence_entropy: 
        max_entropy = max(np.array(re.sub(r"(\[|\])", r"", i).split(', ')).astype(float))
        max_entropy_l.append(max_entropy)
    return max_entropy_l

In [8]:
val_inference_df['max_entropy'] = calc_max_entropy(val_inference_df.sequence_entropy)
test_inference_df['max_entropy'] = calc_max_entropy(test_inference_df.sequence_entropy)

In [10]:
val_inference_df['max_entropy_pct_rank'] = val_inference_df.max_entropy.rank(pct=True)
test_inference_df['max_entropy_pct_rank'] = test_inference_df.max_entropy.rank(pct=True)
# Example of thresholding entropy at 75th percentile
val_inference_df[val_inference_df['max_entropy_pct_rank']>=0.75].min()

question                        Compute the average score of submissions.
real                    SELECT  T1.fname ,  T1.lname FROM Faculty AS T...
pred                    select * from products as t1 join manufacturer...
sequence_entropy        [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
max_entropy                                                      0.604665
max_entropy_pct_rank                                                 0.75
dtype: object

In [54]:

dest_path = 'evaluation/spider_test_75pct_thresh' # do not include file type
eval_df = test_inference_df.merge(pd.DataFrame({'db_id': test_ds['db_id'], 'question': test_ds['question']}), on='question')

write_eval_files(test_inference_df[test_inference_df.max_entropy <= 0.604665], eval_df['db_id'], dest_path)

## Evaluate Error Detection

In [95]:
def print_metrics(df_full, df_err_detect, eval_type):
    true_pos = len(df_full[(df_full.exec_idx.isin(df_err_detect.exec_idx)==False) & (df_full[eval_type]==0)])
    false_pos = len(df_full[(df_full.exec_idx.isin(df_err_detect.exec_idx)==False) & (df_full[eval_type]==1)])
    true_neg = len(df_full[(df_full.exec_idx.isin(df_err_detect.exec_idx)==True) & (df_full[eval_type]==1)])
    false_neg = len(df_full[(df_full.exec_idx.isin(df_err_detect.exec_idx)==True) & (df_full[eval_type]==0)])

    precision = true_pos / (true_pos+false_pos)
    recall = true_pos / (true_pos+false_neg)
    confusion_mat = pd.DataFrame(data=np.array([[true_pos, false_neg], [false_pos, true_neg]]), columns=['positive', 'negative'], index=['positive', 'negative'])
    
    print(f"Error detection precision {eval_type} = {precision:.4f}")
    print(f"Error detection recall {eval_type} = {recall:.4f}")
    print(confusion_mat)


In [97]:
thresh_eval_results = pd.DataFrame(json.load(open('evaluation/turn_scores_spider_test_75pct_thresh.json')))
eval_results =  pd.DataFrame(json.load(open('evaluation/turn_scores_spider_test.json')))

print_metrics(eval_results,thresh_eval_results, 'exec')
print_metrics(eval_results,thresh_eval_results, 'exact')

Error detection precision exec = 0.7411
Error detection recall exec = 0.5818
          positive  negative
positive       352       253
negative       123       306
Error detection precision exact = 0.7832
Error detection recall exact = 0.5776
          positive  negative
positive       372       272
negative       103       287
